# 1. Extracción, Almacenamiento y Seguridad de Datos

Esta sección es responsable de extraer los datos desde la API de CoinGecko, almacenarlos en un archivo CSV y aplicar medidas de seguridad (encriptación y desencriptación de datos).

## Funcionalidades Clave:
### Obtención de Información del Usuario e IP:
Se utiliza os.getlogin() y socket.gethostbyname() para obtener el nombre de usuario e IP de la máquina.
### Carga de Roles y Configuración de Seguridad:
Se carga un archivo JSON (config.json) para gestionar roles y se genera una clave de encriptación usando la librería cryptography con Fernet.
### Extracción de Datos:
Se realiza la llamada a la API de CoinGecko para extraer información sobre criptomonedas y se gestiona la respuesta con manejo de errores y logging.
### Almacenamiento de Datos:
Los datos extraídos se transforman en un DataFrame de Pandas y se guardan en un archivo CSV (datos_cripto.csv).
### Seguridad de Datos:
Se implementa la encriptación del archivo CSV y posteriormente se realiza la desencriptación para verificar el proceso.

In [ ]:
import pandas as pd
import json
import logging
import os
import socket
import requests
from cryptography.fernet import Fernet
from sklearn.preprocessing import MinMaxScaler

# Configuración de logging
logging.basicConfig(
    filename="audit_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Obtención de usuario e IP del sistema
def get_user_info():
    user = os.getlogin()
    ip = socket.gethostbyname(socket.gethostname())
    return user, ip

# Carga del JSON para roles
def load_roles(config_file="config.json"):
    with open(config_file, "r") as f:
        return json.load(f)

roles = load_roles()

# Generar clave de cifrado
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Guardar clave de encriptación
def save_key(key, filename="clave.key"):
    with open(filename, "wb") as key_file:
        key_file.write(key)
    logging.info("Clave de encriptación generada y guardada correctamente.")

save_key(key)

# Función para verificar permisos
def check_access(role):
    if role not in roles:
        logging.warning(f"Acceso denegado: Rol {role} no registrado")
        raise PermissionError("Acceso denegado: Rol no registrado")
    return roles[role]["access_level"]

# Extraer datos desde CoinGecko API
def extract_data_from_api():
    api_url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 10,
        "page": 1,
        "sparkline": False
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        logging.info("Datos extraídos exitosamente desde la API.")
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error al extraer datos de la API: {e}")
        raise

# Guardar datos en CSV
def save_data_to_csv(data, filename="datos_cripto.csv"):
    df = pd.DataFrame([{
        "nombre": coin["name"],
        "simbolo": coin["symbol"],
        "precio_usd": coin["current_price"],
        "cambio_24h": coin["price_change_percentage_24h"],
        "capitalizacion_mercado": coin["market_cap"]
    } for coin in data])

    df.to_csv(filename, index=False)
    logging.info(f"Datos guardados en {filename} correctamente.")
    print(f"Datos guardados en {filename} correctamente.")

# Encriptar datos
def encrypt_file(filename, cipher):
    with open(filename, "rb") as file:
        datos = file.read()
    datos_encriptados = cipher.encrypt(datos)

    with open(f"{filename}.enc", "wb") as file:
        file.write(datos_encriptados)
    logging.info(f"Datos encriptados y guardados en {filename}.enc correctamente.")
    print(f"Datos encriptados y guardados en {filename}.enc correctamente.")

# Desencriptar datos
def decrypt_file(encrypted_filename, decrypted_filename, cipher):
    with open(encrypted_filename, "rb") as file:
        datos_encriptados = file.read()
    datos_desencriptados = cipher.decrypt(datos_encriptados)

    with open(decrypted_filename, "wb") as file:
        file.write(datos_desencriptados)
    logging.info(f"Datos desencriptados y guardados en {decrypted_filename} correctamente.")
    print(f"Datos desencriptados y guardados en {decrypted_filename} correctamente.")

# Ejecución del pipeline de extracción y seguridad
if __name__ == "__main__":
    try:
        data = extract_data_from_api()
        save_data_to_csv(data)
        encrypt_file("datos_cripto.csv", cipher_suite)
        decrypt_file("datos_cripto.csv.enc", "datos_cripto_desencriptados.csv", cipher_suite)
    except Exception as e:
        logging.error(f"Error en el pipeline: {e}")

# 2. Limpieza, Transformación y Modelado de Datos

Esta parte se encarga de preparar y transformar los datos extraídos, y de integrar un modelo de IA (SVM en este caso) para predecir el comportamiento del precio de las criptomonedas.

## Funcionalidades Clave:
### Transformación de Datos:
Se convierte el JSON recibido desde la API en un DataFrame y se exporta a CSV, facilitando la limpieza y la transformación de datos (manejo de duplicados, valores faltantes, etc.).
### Modelado:
Se carga un modelo previamente entrenado (almacenado en svm_model.pkl) y se define una función que realiza la predicción:
Se escalonan los datos usando StandardScaler para adecuar la entrada al modelo.
Se ejecuta la predicción y se retorna el resultado (subida o bajada del precio).


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

# Cargar el modelo entrenado
def load_model():
    # Si has guardado tu modelo en un archivo con joblib, lo puedes cargar aquí
    model = joblib.load("svm_model.pkl")
    return model

# Función para predecir si el precio sube o baja
def predict_price_change(precio_usd, cambio_24h, capitalizacion_mercado, model):
    data = np.array([[precio_usd, cambio_24h, capitalizacion_mercado]])
    # Escalar los datos
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    prediction = model.predict(data_scaled)
    return prediction[0]

# 3. Desarrollo de la Aplicación Web y Documentación

La aplicación web, desarrollada en Streamlit, integra tanto el pipeline de datos como el modelo de IA. Permite al usuario interactuar de forma visual con los resultados.

## Funcionalidades Clave:
### Visualización del Data Pipeline:
Se muestra un diagrama de flujo (utilizando Graphviz) que representa las etapas: Extracción, Limpieza, Almacenamiento, Procesamiento y Modelado.
Se visualizan los logs de procesamiento y se simula la cantidad de registros procesados en cada etapa.
### Exploración y Análisis de Datos:
Se muestran histogramas para visualizar la distribución de variables clave.
Se genera un heatmap para observar las correlaciones entre variables.
Se visualizan los porcentajes de valores faltantes mediante un gráfico de barras.
Se utilizan boxplots para detectar posibles outliers.
### Visualización de Predicciones y Métricas:
Se carga un CSV con columnas como actual y predicho para visualizar la matriz de confusión.
Se muestra la curva ROC/AUC (si se proveen probabilidades).
Se compara visualmente, mediante un gráfico de dispersión, los valores reales y predichos.
Se incluye la opción de calcular y visualizar SHAP values para explicar la predicción del modelo.
### Interactividad:
Permite la carga de nuevos datos a través de la interfaz.
Permite la entrada interactiva de datos para realizar predicciones en vivo.
Se utiliza Plotly para gráficos interactivos que permiten hacer zoom y desplazarse.


In [ ]:
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc
import shap

# Título de la aplicación web
st.title("Clasificador de Cambio de Precio de Criptomonedas")

# Sidebar para navegación entre secciones
st.sidebar.title("Navegación")
page = st.sidebar.radio("Selecciona la sección:", 
                        ["Data Pipeline", "Exploración y Análisis", "Métricas y Predicciones", "Predicción en Vivo", "Carga de Nuevos Datos"])

#############################
# 1. Data Pipeline          #
#############################
if page == "Data Pipeline":
    st.header("Visualización del Data Pipeline")
    
    # Diagrama de flujo usando Graphviz
    pipeline_diagram = """
    digraph {
      Extraccion [label="Extracción"];
      Limpieza [label="Limpieza"];
      Almacenamiento [label="Almacenamiento"];
      Procesamiento [label="Procesamiento"];
      Modelado [label="Modelado"];
      
      Extraccion -> Limpieza -> Almacenamiento -> Procesamiento -> Modelado;
    }
    """
    st.subheader("Diagrama de Flujo")
    st.graphviz_chart(pipeline_diagram)

    # Logs de Procesamiento
    st.subheader("Logs de Procesamiento")
    if os.path.exists("audit_log.txt"):
        with open("audit_log.txt", "r") as f:
            logs = f.read()
        st.text_area("Contenido de audit_log.txt", logs, height=200)
    else:
        st.info("No se encontró el archivo de logs.")

    # Simulación de registros procesados en cada etapa
    st.subheader("Número de Registros Procesados por Etapa")
    try:
        df_pipeline = pd.read_csv("datos_cripto.csv")
        n_registros = len(df_pipeline)
    except Exception:
        n_registros = 10

    pipeline_data = pd.DataFrame({
        "Etapa": ["Extracción", "Limpieza", "Almacenamiento", "Procesamiento", "Modelado"],
        "Registros": [n_registros, n_registros, n_registros, n_registros, n_registros]
    })
    st.bar_chart(pipeline_data.set_index("Etapa"))

#############################
# 2. Exploración y Análisis #
#############################
elif page == "Exploración y Análisis":
    st.header("Exploración y Análisis de Datos")
    try:
        df = pd.read_csv("datos_cripto.csv")
        st.success("Dataset cargado correctamente.")
    except Exception as e:
        st.error(f"Error al cargar el dataset: {e}")
        df = None

    if df is not None:
        st.subheader("Distribución de Datos")
        numeric_cols = ["precio_usd", "cambio_24h", "capitalizacion_mercado"]
        variable = st.selectbox("Selecciona una variable:", numeric_cols)
        fig = px.histogram(df, x=variable, nbins=10, title=f"Histograma de {variable}")
        st.plotly_chart(fig)

        st.subheader("Correlaciones")
        corr = df[numeric_cols].corr()
        fig_corr = px.imshow(corr, text_auto=True, aspect="auto", title="Heatmap de Correlaciones")
        st.plotly_chart(fig_corr)

        st.subheader("Valores Faltantes")
        missing_pct = df.isnull().mean() * 100
        missing_df = missing_pct.reset_index()
        missing_df.columns = ["Variable", "Porcentaje Faltante"]
        fig_missing = px.bar(missing_df, x="Variable", y="Porcentaje Faltante", title="Valores Faltantes (%)")
        st.plotly_chart(fig_missing)

        st.subheader("Detección de Outliers")
        fig_box = go.Figure()
        for col in numeric_cols:
            fig_box.add_trace(go.Box(y=df[col], name=col))
        fig_box.update_layout(title="Boxplots de Variables Numéricas")
        st.plotly_chart(fig_box)

#############################
# 3. Métricas y Predicciones#
#############################
elif page == "Métricas y Predicciones":
    st.header("Visualización de Predicciones y Métricas")
    st.markdown("**Nota:** Para visualizar estas métricas, carga un archivo CSV con las columnas necesarias (por ejemplo, 'actual' y 'predicho').")
    
    uploaded_file = st.file_uploader("Sube un CSV con columnas 'actual' y 'predicho'", type=["csv"])
    if uploaded_file is not None:
        df_metrics = pd.read_csv(uploaded_file)
        st.dataframe(df_metrics.head())

        if "actual" in df_metrics.columns and "predicho" in df_metrics.columns:
            st.subheader("Matriz de Confusión")
            cm = confusion_matrix(df_metrics["actual"], df_metrics["predicho"])
            fig_cm = px.imshow(cm, text_auto=True, 
                               labels=dict(x="Predicción", y="Real"),
                               x=["Negativo", "Positivo"], y=["Negativo", "Positivo"],
                               title="Matriz de Confusión")
            st.plotly_chart(fig_cm)

            if "proba" in df_metrics.columns:
                st.subheader("Curva ROC/AUC")
                fpr, tpr, thresholds = roc_curve(df_metrics["actual"], df_metrics["proba"])
                roc_auc = auc(fpr, tpr)
                fig_roc = go.Figure()
                fig_roc.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC curve'))
                fig_roc.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash')))
                fig_roc.update_layout(title=f"Curva ROC (AUC = {roc_auc:.2f})",
                                      xaxis_title="Tasa de Falsos Positivos",
                                      yaxis_title="Tasa de Verdaderos Positivos")
                st.plotly_chart(fig_roc)
            else:
                st.info("Para la curva ROC/AUC se requiere una columna 'proba' con las probabilidades de predicción.")

            st.subheader("Gráfico de Errores (Actual vs Predicho)")
            fig_error = px.scatter(df_metrics, x="actual", y="predicho", trendline="ols",
                                   title="Comparación entre valores reales y predichos")
            st.plotly_chart(fig_error)
        else:
            st.error("El archivo debe contener las columnas 'actual' y 'predicho'.")

    st.markdown("---")
    st.subheader("SHAP Values")
    st.markdown("Calcular los valores SHAP para explicar la predicción del modelo (usa solo un subconjunto de datos para evitar tiempos prolongados).")
    if st.button("Calcular SHAP"):
        try:
            model = load_model()
            try:
                df_shap = pd.read_csv("datos_cripto.csv")
            except Exception:
                df_shap = pd.DataFrame({
                    "precio_usd": [100, 200, 150],
                    "cambio_24h": [1, -0.5, 0.3],
                    "capitalizacion_mercado": [1000000, 2000000, 1500000]
                })
            X_shap = df_shap[["precio_usd", "cambio_24h", "capitalizacion_mercado"]]
            X_sample = X_shap.head(10)
            explainer = shap.KernelExplainer(model.predict, X_sample)
            shap_values = explainer.shap_values(X_sample)
            st.info("Generando summary plot de SHAP...")
            fig_shap, ax = plt.subplots()
            shap.summary_plot(shap_values, X_sample, show=False)
            st.pyplot(fig_shap.figure)
        except Exception as e:
            st.error(f"Error al calcular SHAP: {e}")
            
    st.markdown("---")
    st.subheader("Importancia de Variables")
    st.markdown("Para modelos basados en árboles se puede mostrar la importancia de variables. Dado que este modelo es SVM, la importancia no es nativa.")
    st.info("La importancia de variables no se muestra para el modelo SVM actual. Para modelos como Random Forest o XGBoost, se podría calcular usando permutation_importance.")

#############################
# 4. Predicción en Vivo     #
#############################
elif page == "Predicción en Vivo":
    st.header("Clasificador de Cambio de Precio de Criptomonedas")
    st.write("Ingresa los valores para predecir si el precio de una criptomoneda subirá o bajará en las próximas 24 horas.")

    precio_usd = st.number_input("Precio en USD", min_value=0.0, value=100.0)
    cambio_24h = st.number_input("Cambio en 24h (%)", min_value=-100.0, value=0.0)
    capitalizacion_mercado = st.number_input("Capitalización de Mercado (en USD)", min_value=0.0, value=1000000.0)

    if st.button("Predecir"):
        try:
            model = load_model()
            prediction = predict_price_change(precio_usd, cambio_24h, capitalizacion_mercado, model)
            if prediction == 1:
                st.success("El precio de la criptomoneda subirá.")
            else:
                st.error("El precio de la criptomoneda bajará.")
            st.write("### Datos de entrada para la predicción:")
            st.write(f"**Precio en USD:** {precio_usd}")
            st.write(f"**Cambio en 24h:** {cambio_24h}%")
            st.write(f"**Capitalización de mercado:** {capitalizacion_mercado} USD")
        except Exception as e:
            st.error(f"Error al realizar la predicción: {e}")

#############################
# 5. Carga de Nuevos Datos  #
#############################
elif page == "Carga de Nuevos Datos":
    st.header("Carga de Nuevos Datos")
    uploaded_csv = st.file_uploader("Sube un archivo CSV", type=["csv"])
    if uploaded_csv is not None:
        try:
            df_nuevos = pd.read_csv(uploaded_csv)
            st.success("Archivo cargado exitosamente.")
            st.dataframe(df_nuevos.head())
        except Exception as e:
            st.error(f"Error al cargar el archivo: {e}")
    else:
        st.info("Espera a subir un archivo CSV para visualizar los datos.")